In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
device


In [ ]:
do_download:bool=False
if do_download:
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape


## Different methods to compute rolling mean on T axis.
like weighted sum with same weight on previous items of sequence

In [ ]:
# method 1
xbow = torch.zeros((B, T, C))
xbow.shape
for b in range(B):
    for t in range(T):
        xprev = x[b, : t + 1]
        # print(b, t, xprev.shape)
        xbow[b, t] = torch.mean(xprev, 0)
        # print("-----")
        # print(xbow)

xbow.shape


In [ ]:
# method 2
wei = torch.tril(torch.ones((T, T)))
# print(wei)
wei = wei / wei.sum(axis=1, keepdim=True)
print("weight", wei)
xbow2 = wei @ x  # B,T,C @ B,T,T ---> B,T,C
torch.allclose(xbow, xbow2)


In [ ]:
# method 3 using softmax
tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
# print(wei)
wei = wei.masked_fill(tril == 0, value=float("-inf"))
# print(wei)
wei = F.softmax(wei, dim=1)
print("weight", wei)

xbow3 = wei @ x
torch.allclose(xbow, xbow3)


# model

In [ ]:
with open("./input.txt", "r", encoding="utf-8") as f:
    text = f.read()


In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for i, s in enumerate(chars)}


def encode(s: str) -> list[int]:
    return [stoi[c] for c in s]


def decode(l: list[int]) -> str:
    return "".join([itos[i] for i in l])


In [ ]:
decode(encode("j'aime les chats"))
# stoi["a"], stoi["A"]


In [ ]:
# hyperparams
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-3

eval_iters = 200
vocab_size = 65
n_embd = 32
num_heads = 4


In [ ]:
torch.manual_seed(42)
# Compute train and validation data
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(text))
train = data[:n]
val = data[n:]


def get_batch(split: str = "train"):
    data = train if split == "train" else val
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # ix are random starting points of each batch
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x, y


In [ ]:
x, y = get_batch()
x, y
# x.shape, y.shape


In [ ]:
torch.manual_seed(42)
B, T, C = 16, 8, 32
x = torch.rand(B, T, C)

head_size = 32
key = nn.Linear(C, head_size)
k = key(x)
query = nn.Linear(C, head_size)
q = query(x)
value = nn.Linear(C, head_size)
v = value(x)

wei = q @ k.transpose(-2, -1) * head_size**-0.5
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)

out = wei @ v
# out.shape


In [ ]:
# test with a hard coded query and key
q = torch.tensor([[[0.0, 1.0], [2.0, 3.0], [4.0, 5.0]]])
k = torch.tensor([[[0.0, 1.0], [2.0, 3.0], [4.0, 5.0]]])
B, T, C = q.shape
q.shape
wei = q @ k.transpose(-2, -1)  # * head_size**-0.5
tril = torch.tril(torch.ones(T, T))
# wei = wei.masked_fill(tril == 0, float("-inf"))
# wei = F.softmax(wei, dim=-1)
wei


In [ ]:
class Head(nn.Module):
    def __init__(self, head_size: int) -> None:
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", tensor=torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        v = self.value(x)
        out = wei @ v
        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads: int, head_size: int) -> None:
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1)

class FeedForward(nn.Module):
    def __init__(self, n_embd: int) -> None:
        super().__init__()
        self.net = nn.Sequential([nn.Linear(n_embd,n_embd), nn.ReLU()])
        
    def forward(self, x):
        return self.net(x)


class BigramModel(nn.Module):
    def __init__(self, vocab_size: int) -> None:
        super().__init__()
        self.token_emb = torch.nn.Embedding(vocab_size, n_embd)
        self.pos_emb = torch.nn.Embedding(block_size, n_embd)
        self.sa_head = MultiHeadAttention(num_heads=num_heads, head_size=n_embd // num_heads)
        self.lm_head = torch.nn.Linear(n_embd, vocab_size)
        # self.lm_head = Head(head_size=1)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_emb(idx)
        pos_emb = self.pos_emb(torch.arange(T, device=device))
        # print("pos_emb shape", pos_emb.shape)
        x = tok_emb + pos_emb  # B, T, C
        # print(f"x shape: {x.shape}")
        x = self.sa_head(x)
        logits = self.lm_head(x)
        
        # print(f"logits shape: {logits.shape}")
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            # print("generate logit shape", logits.shape)

            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx


model = BigramModel(vocab_size)
model = model.to(device)
xb, yb = get_batch("train")
train_logit, train_loss = model(xb)


In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


max_iters = 3000
eval_interval = 500

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

model_id = "cop_gpt"
model_version = "0.1"
log_dir = Path("../runs") / f"{model_id}-{model_version}" / f"{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"

writer = SummaryWriter(log_dir=log_dir)

for i in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if i % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    for name, weight in model.named_parameters():
        writer.add_histogram(name, weight, i)

    optimizer.step()


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=200)[0].tolist()))


In [ ]:
params = []
for n, p in model.named_parameters():
    print(n, p.shape)
    params.append((n, p.data))


In [ ]:
npp = params[1][1].cpu().detach().numpy()

plt.imshow(npp, cmap='hot', interpolation='nearest')
plt.show()

### view layer values